In [1]:
# -*- coding: utf-8 -*- for 24 MONTHS
# Colab: Upload ONE merged CSV (24+ rows) → produce ONE SANITIZED LLM-ready YAML (no imputation) and download it.
# Sanitization: no original filename, no calendar dates/timeline in the YAML.

import io, os, re, math, json, uuid
import numpy as np
import pandas as pd

# ---- Colab upload helper
try:
    from google.colab import files as _files
except Exception as e:
    raise RuntimeError("This cell is intended to run in Google Colab.") from e

# =========================
# Config
# =========================
WINDOW_LEN   = 24
TAKE_LAST_24 = True     # True: use last 24 rows after sorting by date; False: first 24
# If denominator is zero in percent-change, we drop that return (no error, no imputation).

# Canonical indicator keys and flexible aliases (case-insensitive)
INDICATOR_ALIASES = {
    "CPI":       [r"^cpi$"],
    "PPI":       [r"^ppi$"],
    "FEDFUNDS":  [r"^fedfunds$|^federal[_\s]?funds|^ffr$|^policy[_\s]?rate$"],
    "DGS10":     [r"^dgs?10$|^10[-_\s]?year|^10y|^ust10"],
    "SP500_PE":  [r"^sp500[_\s]?pe$|^pe$|^p\/?e$"],
    "DJIA":      [r"^djia$|^dow|^dow[_\s]?jones|^djia[_\s]?close$"],
}
DATE_HINTS = [r"^date$", r"^month$", r"^time$", r"^period$"]

# =========================
# Helpers (no imputation)
# =========================
def _match_column(name, patterns):
    n = name.strip().lower()
    for pat in patterns:
        if re.search(pat, n, flags=re.I):
            return True
    return False

def _find_date_col(df):
    # Try named hints
    for c in df.columns:
        if _match_column(str(c), DATE_HINTS):
            try:
                _ = pd.to_datetime(df[c], errors="raise")
                return c
            except Exception:
                pass
    # Fallback: first datetime-like
    for c in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[c]):
            return c
    return None

def _normalize_columns(df):
    # map canonical indicator -> column name (or None)
    mapping = {}
    cols = list(df.columns)
    for key, pats in INDICATOR_ALIASES.items():
        found = None
        for c in cols:
            if pd.api.types.is_numeric_dtype(df[c]) and _match_column(str(c), pats):
                found = c; break
        mapping[key] = found
    return mapping

def _levels_24(df, colname):
    # Return a float array of length 24 with NaNs preserved (no imputation)
    s = pd.to_numeric(df[colname], errors="coerce").astype(float)
    vals = s.to_numpy()
    if len(vals) < WINDOW_LEN:
        raise ValueError(f"{colname}: only {len(vals)} rows available (need >= {WINDOW_LEN}).")
    return vals[-WINDOW_LEN:] if TAKE_LAST_24 else vals[:WINDOW_LEN]

def _pct_changes_with_nans(levels):
    # r_t = 100 * (x_t - x_{t-1}) / x_{t-1}; if either side is NaN or x_{t-1}==0 → NaN
    x = np.asarray(levels, dtype=float)
    r = np.full(WINDOW_LEN-1, np.nan, dtype=float)
    for t in range(1, len(x)):
        a, b = x[t-1], x[t]
        if np.isfinite(a) and np.isfinite(b) and (a != 0.0):
            r[t-1] = 100.0 * (b - a) / a
    return r  # length 23, may contain NaNs

def _ols_slope_and_r2_with_missing(y):
    # Use only valid points; regress y on original time indices where valid.
    idx = np.where(np.isfinite(y))[0]
    if len(idx) < 3:
        return np.nan, np.nan
    t = (idx + 1).astype(float)  # 1..23 on valid slots
    yv = y[idx].astype(float)
    t_mean, y_mean = t.mean(), yv.mean()
    cov = np.sum((t - t_mean) * (yv - y_mean))
    var_t = np.sum((t - t_mean) ** 2)
    beta = cov / var_t if var_t != 0 else np.nan
    alpha = y_mean - beta * t_mean if np.isfinite(beta) else np.nan
    if not np.isfinite(beta) or not np.isfinite(alpha):
        return np.nan, np.nan
    y_hat = alpha + beta * t
    ssr = np.sum((yv - y_hat) ** 2)
    sst = np.sum((yv - y_mean) ** 2)
    r2 = np.nan if sst == 0 else (1.0 - ssr / sst)
    return float(beta), float(r2)

def _std_sample_valid(a):
    v = a[np.isfinite(a)]
    return float(np.std(v, ddof=1)) if len(v) >= 2 else np.nan

def _share_up(a):
    v = a[np.isfinite(a)]
    if len(v) == 0:
        return np.nan
    return float(np.mean(v > 0.0))

def _std_subset(a, start_idx, end_idx):
    # a indices inclusive of [start_idx, end_idx]; both 0-based on returns array (length 23)
    sub = a[start_idx:end_idx+1]
    return _std_sample_valid(sub)

def _contiguous_runs_valid(levels):
    x = np.asarray(levels, dtype=float)
    runs = []
    start = None
    for i, v in enumerate(x):
        if np.isfinite(v):
            if start is None:
                start = i
        else:
            if start is not None:
                runs.append((start, i-1))
                start = None
    if start is not None:
        runs.append((start, len(x)-1))
    return runs

def _max_drawup_on_runs(levels):
    x = np.asarray(levels, dtype=float)
    best_up, best_peak = -np.inf, None
    for s, e in _contiguous_runs_valid(x):
        if e - s + 1 < 2:
            continue
        min_so_far = x[s]; t_peak_local = s
        up_local_best = 0.0
        for j in range(s+1, e+1):
            if x[j] / min_so_far - 1.0 > up_local_best:
                up_local_best = x[j] / min_so_far - 1.0
                t_peak_local = j
            if x[j] < min_so_far:
                min_so_far = x[j]
        if up_local_best > best_up:
            best_up = up_local_best
            best_peak = t_peak_local
    if best_peak is None:
        return np.nan, np.nan
    return 100.0 * best_up, int(best_peak + 1)  # 1-based index

def _max_drawdown_on_runs(levels):
    x = np.asarray(levels, dtype=float)
    best_down, best_trough = +np.inf, None  # most negative %
    for s, e in _contiguous_runs_valid(x):
        if e - s + 1 < 2:
            continue
        max_so_far = x[s]; t_trough_local = s
        down_local_best = 0.0
        for j in range(s+1, e+1):
            d = (x[j] / max_so_far) - 1.0
            if d < down_local_best:
                down_local_best = d
                t_trough_local = j
            if x[j] > max_so_far:
                max_so_far = x[j]
        if down_local_best < best_down:
            best_down = down_local_best
            best_trough = t_trough_local
    if best_trough is None:
        return np.nan, np.nan
    return 100.0 * best_down, int(best_trough + 1)

def _acf1_with_missing(y):
    if len(y) < 2:
        return np.nan
    y0 = y[:-1]; y1 = y[1:]
    mask = np.isfinite(y0) & np.isfinite(y1)
    if mask.sum() < 2:
        return np.nan
    a = y0[mask] - y0[mask].mean()
    b = y1[mask] - y1[mask].mean()
    denom = math.sqrt(np.sum(a*a) * np.sum(b*b))
    return float(0.0 if denom == 0 else np.sum(a*b) / denom)

def _fmt_pct_or_uncertain(x, nd=1):
    return f"{x:+.{nd}f}" if np.isfinite(x) else "uncertain"

def _fmt_share_or_uncertain(x, nd=2):
    return f"{x:.{nd}f}" if np.isfinite(x) else "uncertain"

def summarize_indicator_levels_no_impute(levels):
    x = np.asarray(levels, dtype=float)
    if len(x) != WINDOW_LEN:
        raise ValueError(f"Expected {WINDOW_LEN} monthly levels; got {len(x)}.")
    r = _pct_changes_with_nans(x)
    net_change = np.nan
    if np.isfinite(x[0]) and np.isfinite(x[-1]) and x[0] != 0.0:
        net_change = 100.0 * (x[-1]/x[0] - 1.0)
    slope, r2 = _ols_slope_and_r2_with_missing(r)
    up_share  = _share_up(r)
    vol_std   = _std_sample_valid(r)
    std_early = _std_subset(r, 1, 7)   # r_2..r_8 -> indices 1..7
    std_late  = _std_subset(r, 17, 23) # r_18..r_24 -> 17..23
    vol_late_minus_early = (std_late - std_early) if (np.isfinite(std_early) and np.isfinite(std_late)) else np.nan
    max_up,  t_peak   = _max_drawup_on_runs(x)
    max_down, t_trough = _max_drawdown_on_runs(x)
    acf1 = _acf1_with_missing(r)
    return {
        "net_change_24m_pct":       _fmt_pct_or_uncertain(net_change, 1),
        "slope_ols_pct_per_mo":     _fmt_pct_or_uncertain(slope, 2),
        "trend_r2":                 _fmt_share_or_uncertain(r2, 2),
        "up_month_share":           _fmt_share_or_uncertain(up_share, 2),
        "vol_std_pct":              _fmt_pct_or_uncertain(vol_std, 1),
        "vol_late_minus_early_pct": _fmt_pct_or_uncertain(vol_late_minus_early, 1),
        "max_drawup_pct":           _fmt_pct_or_uncertain(max_up, 1),
        "t_peak":                   int(t_peak) if np.isfinite(t_peak) else "uncertain",
        "max_drawdown_pct":         _fmt_pct_or_uncertain(max_down, 1),
        "t_trough":                 int(t_trough) if np.isfinite(t_trough) else "uncertain",
        "acf1":                     _fmt_share_or_uncertain(acf1, 2),
    }

# =========================
# Main: upload one CSV, process (no impute), and download SANITIZED YAML
# =========================
print("Upload ONE merged CSV (e.g., Merged_*_Last_24_months.csv)…")
uploaded = _files.upload()
if not uploaded:
    raise SystemExit("No file uploaded.")

# Take the first uploaded file
in_name = list(uploaded.keys())[0]
raw = uploaded[in_name]
df = pd.read_csv(io.BytesIO(raw))

# Sort by date if present (for windowing only; we do NOT emit dates)
dcol = _find_date_col(df)
if dcol is not None:
    df = df.copy()
    df[dcol] = pd.to_datetime(df[dcol], errors="coerce")
    df = df.dropna(subset=[dcol]).sort_values(dcol, ascending=True, kind="mergesort").reset_index(drop=True)
else:
    df = df.reset_index(drop=True)

# Restrict deterministically to a 24-row window
if len(df) < WINDOW_LEN:
    raise ValueError(f"File has only {len(df)} rows; need at least {WINDOW_LEN}.")
df_24 = df.tail(WINDOW_LEN) if TAKE_LAST_24 else df.head(WINDOW_LEN)

# Detect indicators present
mapping = _normalize_columns(df_24)
present = [k for k,v in mapping.items() if v is not None]
if not present:
    raise ValueError("No recognizable indicator columns found (CPI, PPI, FEDFUNDS, DGS10, SP500_PE, DJIA).")

# Build NL summaries (no imputation) and track missing months (indices only)
results = {}
missing_map = {}
for key in present:
    col = mapping[key]
    levels = _levels_24(df_24, col)  # preserves NaNs
    results[key] = summarize_indicator_levels_no_impute(levels)
    miss_idx = [i+1 for i,v in enumerate(levels) if not np.isfinite(v)]
    if miss_idx:
        missing_map[key] = miss_idx

# Compose SANITIZED JSON (no filename, no dates)
anon_id = f"anon_{uuid.uuid4().hex[:8]}"
out_json = f"{anon_id}_window_llm_noimpute.json"

output = {
    "window": "24_months",
    "indicators": results
}
if missing_map:
    output["missing_data"] = missing_map

with open(out_json, "w", encoding="utf-8") as f:
    json.dump(output, f, indent=2)

print(f"\nIndicators detected: {present}")
if missing_map:
    print("Missing months (1..24) by indicator:", missing_map)
print(f"Created JSON (sanitized): {out_json}")

# Auto-download to your local laptop
_files.download(out_json)

Upload ONE merged CSV (e.g., Merged_*_Last_24_months.csv)…


Saving Merged_Subprime_Bubble.csv to Merged_Subprime_Bubble.csv

Indicators detected: ['CPI', 'PPI', 'FEDFUNDS', 'DGS10', 'SP500_PE', 'DJIA']
Missing months (1..24) by indicator: {'DGS10': [18]}
Created JSON (sanitized): anon_4612e7e0_window_llm_noimpute.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
# -*- coding: utf-8 -*-
# Colab: Upload ONE merged CSV (>=2 rows) → produce ONE SANITIZED LLM-ready JSON (no imputation) and download it.
# Sanitization: no original filename, no calendar dates/timeline in the JSON.
# This version processes the full dataset, not a fixed-size window.

import io, os, re, math, json, uuid
import numpy as np
import pandas as pd

# ---- Colab upload helper
try:
    from google.colab import files as _files
except Exception as e:
    raise RuntimeError("This cell is intended to run in Google Colab.") from e

# =========================
# Config
# =========================
# Canonical indicator keys and flexible aliases (case-insensitive)
INDICATOR_ALIASES = {
    "CPI":       [r"^cpi$"],
    "PPI":       [r"^ppi$"],
    "FEDFUNDS":  [r"^fedfunds$|^federal[_\s]?funds|^ffr$|^policy[_\s]?rate$"],
    "DGS10":     [r"^dgs?10$|^10[-_\s]?year|^10y|^ust10"],
    "SP500_PE":  [r"^sp500[_\s]?pe$|^pe$|^p\/?e$"],
    "DJIA":      [r"^djia$|^dow|^dow[_\s]?jones|^djia[_\s]?close$"],
}
DATE_HINTS = [r"^date$", r"^month$", r"^time$", r"^period$"]

# =========================
# Helpers (no imputation)
# =========================
def _match_column(name, patterns):
    n = name.strip().lower()
    for pat in patterns:
        if re.search(pat, n, flags=re.I):
            return True
    return False

def _find_date_col(df):
    # Try named hints
    for c in df.columns:
        if _match_column(str(c), DATE_HINTS):
            try:
                _ = pd.to_datetime(df[c], errors="raise")
                return c
            except Exception:
                pass
    # Fallback: first datetime-like
    for c in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[c]):
            return c
    return None

def _normalize_columns(df):
    # map canonical indicator -> column name (or None)
    mapping = {}
    cols = list(df.columns)
    for key, pats in INDICATOR_ALIASES.items():
        found = None
        for c in cols:
            if pd.api.types.is_numeric_dtype(df[c]) and _match_column(str(c), pats):
                found = c; break
        mapping[key] = found
    return mapping

def _levels_all(df, colname):
    # Return a float array with NaNs preserved (no imputation), using ALL available rows
    s = pd.to_numeric(df[colname], errors="coerce").astype(float)
    vals = s.to_numpy()
    if len(vals) < 2:
        raise ValueError(f"{colname}: need at least 2 rows to compute returns; got {len(vals)}.")
    return vals

def _pct_changes_with_nans(levels):
    # r_t = 100 * (x_t - x_{t-1}) / x_{t-1}; if either side is NaN or x_{t-1}==0 → NaN
    x = np.asarray(levels, dtype=float)
    R = max(0, len(x) - 1)
    r = np.full(R, np.nan, dtype=float)
    for t in range(1, len(x)):
        a, b = x[t-1], x[t]
        if np.isfinite(a) and np.isfinite(b) and (a != 0.0):
            r[t-1] = 100.0 * (b - a) / a
    return r  # length = len(levels) - 1, may contain NaNs

def _ols_slope_and_r2_with_missing(y):
    # Use only valid points; regress y on original time indices where valid.
    idx = np.where(np.isfinite(y))[0]
    if len(idx) < 3:
        return np.nan, np.nan
    t = (idx + 1).astype(float)  # 1..R on valid slots
    yv = y[idx].astype(float)
    t_mean, y_mean = t.mean(), yv.mean()
    cov = np.sum((t - t_mean) * (yv - y_mean))
    var_t = np.sum((t - t_mean) ** 2)
    beta = cov / var_t if var_t != 0 else np.nan
    alpha = y_mean - beta * t_mean if np.isfinite(beta) else np.nan
    if not np.isfinite(beta) or not np.isfinite(alpha):
        return np.nan, np.nan
    y_hat = alpha + beta * t
    ssr = np.sum((yv - y_hat) ** 2)
    sst = np.sum((yv - y_mean) ** 2)
    r2 = np.nan if sst == 0 else (1.0 - ssr / sst)
    return float(beta), float(r2)

def _std_sample_valid(a):
    v = a[np.isfinite(a)]
    return float(np.std(v, ddof=1)) if len(v) >= 2 else np.nan

def _share_up(a):
    v = a[np.isfinite(a)]
    if len(v) == 0:
        return np.nan
    return float(np.mean(v > 0.0))

def _contiguous_runs_valid(levels):
    x = np.asarray(levels, dtype=float)
    runs = []
    start = None
    for i, v in enumerate(x):
        if np.isfinite(v):
            if start is None:
                start = i
        else:
            if start is not None:
                runs.append((start, i-1))
                start = None
    if start is not None:
        runs.append((start, len(x)-1))
    return runs

def _max_drawup_on_runs(levels):
    x = np.asarray(levels, dtype=float)
    best_up, best_peak = -np.inf, None
    for s, e in _contiguous_runs_valid(x):
        if e - s + 1 < 2:
            continue
        min_so_far = x[s]; t_peak_local = s
        up_local_best = 0.0
        for j in range(s+1, e+1):
            if x[j] / min_so_far - 1.0 > up_local_best:
                up_local_best = x[j] / min_so_far - 1.0
                t_peak_local = j
            if x[j] < min_so_far:
                min_so_far = x[j]
        if up_local_best > best_up:
            best_up = up_local_best
            best_peak = t_peak_local
    if best_peak is None:
        return np.nan, np.nan
    return 100.0 * best_up, int(best_peak + 1)  # 1-based index

def _max_drawdown_on_runs(levels):
    x = np.asarray(levels, dtype=float)
    best_down, best_trough = +np.inf, None  # most negative %
    for s, e in _contiguous_runs_valid(x):
        if e - s + 1 < 2:
            continue
        max_so_far = x[s]; t_trough_local = s
        down_local_best = 0.0
        for j in range(s+1, e+1):
            d = (x[j] / max_so_far) - 1.0
            if d < down_local_best:
                down_local_best = d
                t_trough_local = j
            if x[j] > max_so_far:
                max_so_far = x[j]
        if down_local_best < best_down:
            best_down = down_local_best
            best_trough = t_trough_local
    if best_trough is None:
        return np.nan, np.nan
    return 100.0 * best_down, int(best_trough + 1)

def _acf1_with_missing(y):
    if len(y) < 2:
        return np.nan
    y0 = y[:-1]; y1 = y[1:]
    mask = np.isfinite(y0) & np.isfinite(y1)
    if mask.sum() < 2:
        return np.nan
    a = y0[mask] - y0[mask].mean()
    b = y1[mask] - y1[mask].mean()
    denom = math.sqrt(np.sum(a*a) * np.sum(b*b))
    return float(0.0 if denom == 0 else np.sum(a*b) / denom)

def _fmt_pct_or_uncertain(x, nd=1):
    return f"{x:+.{nd}f}" if np.isfinite(x) else "uncertain"

def _fmt_share_or_uncertain(x, nd=2):
    return f"{x:.{nd}f}" if np.isfinite(x) else "uncertain"

def _std_early_late_general(r):
    """
    Compute early/late volatility on a percentage basis.
    The window size for early/late volatility is scaled based on the total
    length of the data series. This ensures a consistent comparison even
    with different dataset sizes.
    Returns (std_early, std_late) with NaN → 'uncertain'.
    """
    R = len(r)
    if R < 2:
        return np.nan, np.nan
    if R >= 23:
        early_start, early_end = 1, 7
        late_start,  late_end  = R - 7, R - 1
    else:
        k = max(2, int(round(7 * R / 23.0)))
        early_start = 1 if R > 1 else 0
        early_end   = min(R - 1, early_start + k - 1)
        late_start  = max(0, R - k)
        late_end    = R - 1
    std_early = _std_sample_valid(r[early_start:early_end+1])
    std_late  = _std_sample_valid(r[late_start:late_end+1])
    return std_early, std_late

def summarize_indicator_levels_no_impute(levels):
    x = np.asarray(levels, dtype=float)
    N = len(x)
    if N < 2:
        raise ValueError(f"Expected at least 2 rows; got {N}.")
    r = _pct_changes_with_nans(x)
    # Net change over the whole available window
    net_change = np.nan
    if np.isfinite(x[0]) and np.isfinite(x[-1]) and x[0] != 0.0:
        net_change = 100.0 * (x[-1]/x[0] - 1.0)
    slope, r2 = _ols_slope_and_r2_with_missing(r)
    up_share  = _share_up(r)
    vol_std   = _std_sample_valid(r)
    std_early, std_late = _std_early_late_general(r)
    vol_late_minus_early = (std_late - std_early) if (np.isfinite(std_early) and np.isfinite(std_late)) else np.nan
    max_up,  t_peak   = _max_drawup_on_runs(x)
    max_down, t_trough = _max_drawdown_on_runs(x)
    acf1 = _acf1_with_missing(r)
    return {
        # Keep original field names & "uncertain" formatting
        "net_change_pct":           _fmt_pct_or_uncertain(net_change, 1),
        "slope_ols_pct_per_mo":     _fmt_pct_or_uncertain(slope, 2),
        "trend_r2":                 _fmt_share_or_uncertain(r2, 2),
        "up_month_share":           _fmt_share_or_uncertain(up_share, 2),
        "vol_std_pct":              _fmt_pct_or_uncertain(vol_std, 1),
        "vol_late_minus_early_pct": _fmt_pct_or_uncertain(vol_late_minus_early, 1),
        "max_drawup_pct":           _fmt_pct_or_uncertain(max_up, 1),
        "t_peak":                   int(t_peak) if np.isfinite(t_peak) else "uncertain",
        "max_drawdown_pct":         _fmt_pct_or_uncertain(max_down, 1),
        "t_trough":                 int(t_trough) if np.isfinite(t_trough) else "uncertain",
        "acf1":                     _fmt_share_or_uncertain(acf1, 2),
    }

# =========================
# Main: upload one CSV, process (no impute), and download SANITIZED JSON
# =========================
print("Upload ONE merged CSV (e.g., Merged_*.csv)…")
uploaded = _files.upload()
if not uploaded:
    raise SystemExit("No file uploaded.")

# Take the first uploaded file
in_name = list(uploaded.keys())[0]
raw = uploaded[in_name]
df = pd.read_csv(io.BytesIO(raw))

# Sort by date if present (for ordering only; we do NOT emit dates)
dcol = _find_date_col(df)
if dcol is not None:
    df = df.copy()
    df[dcol] = pd.to_datetime(df[dcol], errors="coerce")
    df = df.dropna(subset=[dcol]).sort_values(dcol, ascending=True, kind="mergesort").reset_index(drop=True)
else:
    df = df.reset_index(drop=True)

# Require at least 2 rows overall (no fixed 24-row slice)
if len(df) < 2:
    raise ValueError(f"File has only {len(df)} rows; need at least 2.")

# Detect indicators present over ALL rows
mapping = _normalize_columns(df)
present = [k for k,v in mapping.items() if v is not None]
if not present:
    raise ValueError("No recognizable indicator columns found (CPI, PPI, FEDFUNDS, DGS10, SP500_PE, DJIA).")

# Build NL summaries (no imputation) and track missing indices (1-based)
results = {}
missing_map = {}
for key in present:
    col = mapping[key]
    levels = _levels_all(df, col)  # preserves NaNs across ALL rows
    results[key] = summarize_indicator_levels_no_impute(levels)
    miss_idx = [i+1 for i,v in enumerate(levels) if not np.isfinite(v)]
    if miss_idx:
        missing_map[key] = miss_idx

# Compose SANITIZED JSON (no filename, no dates)
anon_id = f"anon_{uuid.uuid4().hex[:8]}"
out_json = f"{anon_id}_window_llm_noimpute.json"

output = {
    # Keep the same key name but clarify it's the whole series
    "window": "full_series_rows",
    "indicators": results
}
if missing_map:
    output["missing_data"] = missing_map  # same key name as original

with open(out_json, "w", encoding="utf-8") as f:
    json.dump(output, f, indent=2)

print(f"\nIndicators detected: {present}")
if missing_map:
    print("Missing months (1..N) by indicator:", missing_map)
print(f"Created JSON (sanitized): {out_json}")

# Auto-download to your local laptop
_files.download(out_json)


Upload ONE merged CSV (e.g., Merged_*.csv)…


Saving 2001.04-2003.12.csv to 2001.04-2003.12.csv

Indicators detected: ['CPI', 'PPI', 'FEDFUNDS', 'DGS10', 'SP500_PE', 'DJIA']
Created JSON (sanitized): anon_d6008fd9_window_llm_noimpute.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>